In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# 讀取 CSV
df = pd.read_csv("data.csv")

# 初始化中文向量化模型
model = SentenceTransformer('shibing624/text2vec-base-chinese')

# 合併重要字段成檢索文本
df['search_text'] = df['商品名稱'] + " " + df['描述'] + " " + df['商品標籤']
df['search_text'] = df['search_text'].fillna("")  # 將 NaN 替換為空字串
df['search_text'] = df['search_text'].astype(str)


# 向量化處理
embeddings = model.encode(df['search_text'].tolist())


In [4]:
import faiss
import numpy as np

# 建立向量索引
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))  # 加入商品向量

# 保存索引 (可選)
faiss.write_index(index, "product_index.faiss")


In [26]:
def search_products(query, top_k=5):
    # 查詢向量化
    query_embedding = model.encode([query])
    
    print('query_embedding: ', end='')
    print(query_embedding.shape)

    # 檢索最近的商品
    distances, indices = index.search(np.array(query_embedding), top_k)
    
    # 返回結果
    results = df.iloc[indices[0]]
    return results[['分類', '商品名稱', '描述', '價格']], distances

# 測試查詢
query = "滑鼠"
results = search_products(query)
print(results)


query_embedding: (1, 768)
(     分類                                   商品名稱  \
100  滑鼠                            羅技 M90 光學滑鼠   
94   滑鼠                         羅技 G102 炫彩遊戲滑鼠   
97   滑鼠                         羅技 M190 無線滑鼠-黑   
96   滑鼠  羅技 M100r 光學滑鼠 有線設計 雙手均適用 1000 dpi高解析度   
99   滑鼠                         羅技 M190 無線滑鼠-紅   

                                                    描述     價格  
100              ['◆羅技 logi M90', '◆全尺寸左右手皆適用', '◆有線']  189 元  
94   ['◆七彩光學效果，讓遊戲更充滿樂趣', '◆高達8000DPI遊戲等級感應器', '◆機械...  499 元  
97           ['◆全尺寸的舒適感', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']  349 元  
96   ['◆有線設計，免除更換電池的過程', '◆雙手均適用的舒適設計', '◆1000 dpi高...  299 元  
99           ['◆全尺寸的舒適感', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']  349 元  , array([[347.71057, 356.52734, 371.08612, 383.7093 , 384.15964]],
      dtype=float32))


In [11]:
query = "電競滑鼠"
results, distances = search_products(query)
results

,分類,商品名稱,描述,價格
94,滑鼠,羅技 G102 炫彩遊戲滑鼠,"['◆七彩光學效果，讓遊戲更充滿樂趣', '◆高達8000DPI遊戲等級感應器', '◆機械...",499 元
2794,電腦應用週邊,MS-G2 電擊新安洲遊戲滑鼠,"['', '', '']",價格須至店內詢問
100,滑鼠,羅技 M90 光學滑鼠,"['◆羅技 logi M90', '◆全尺寸左右手皆適用', '◆有線']",189 元
95,滑鼠,羅技 G304 無線電競滑鼠-黑,"['◆高效能 HERO 感應器，高達10倍省電效能', '◆超疾速 LIGHTSPEED 無...",1090 元
2795,電腦應用週邊,MS-G3 電擊剎帝利遊戲滑鼠,"['', '', '']",價格須至店內詢問


In [12]:
distances

array([[227.5427 , 250.2357 , 254.87384, 258.8756 , 260.0652 ]],
      dtype=float32)

In [13]:
query = "王益文"
results, distances = search_products(query)
results

,分類,商品名稱,描述,價格
2673,電子套件,MTLAPC16128 邏輯分析儀(數位電子乙級技能檢定)LAP-C(16128),"['', '', '']",價格須至店內詢問
3241,電腦產品,【A4 TECH 】BLOODY V7+B2-5 血手系列+血手寶典(含特血核心三/四）/光學滑鼠,"['◆BLOODY 血手令 血手幽靈 V7 智能多核 左三槍 USB 有線 電競遊戲滑鼠',...",899 元
1219,通訊電子產品,CP-376M 4/6/8P三用壓接工具,"['', '', '']",價格須至店內詢問
1338,通訊電子產品,LS-920E UY接續子,"['', '', '']",價格須至店內詢問
1081,一般電子零件,AS001 MINI 轉接頭 15公15母 三排 90度,"['', '', '']",價格須至店內詢問


In [14]:
distances

array([[340.2034 , 341.17004, 344.2594 , 345.12506, 345.16287]],
      dtype=float32)

In [19]:
query = "神奇海螺"
results, distances = search_products(query)
results

,分類,商品名稱,描述,價格
2713,電腦應用週邊,EMS102BK E-BLUE藍光鯊魚鼠(黑),"['', '', '']",價格須至店內詢問
2715,電腦應用週邊,EMS102GR E-BLUE藍光鯊魚鼠(綠),"['', '', '']",價格須至店內詢問
2717,電腦應用週邊,EMS102WH E-BLUE藍光鯊魚鼠(白),"['', '', '']",價格須至店內詢問
3016,汽機車用品,8093A 1156-18食人魚LED 12V,"['顏色:紅.黃.綠.藍.白色', '', '']",價格須至店內詢問
3018,汽機車用品,8094A 1157-18食人魚LED 12V,"['顏色:紅.黃.綠.藍.白色', '', '']",價格須至店內詢問


In [20]:
distances

array([[312.02997, 316.02222, 320.35962, 335.82098, 340.3968 ]],
      dtype=float32)

In [21]:
query = "羅技滑鼠"
results, distances = search_products(query)
results

,分類,商品名稱,描述,價格
94,滑鼠,羅技 G102 炫彩遊戲滑鼠,"['◆七彩光學效果，讓遊戲更充滿樂趣', '◆高達8000DPI遊戲等級感應器', '◆機械...",499 元
100,滑鼠,羅技 M90 光學滑鼠,"['◆羅技 logi M90', '◆全尺寸左右手皆適用', '◆有線']",189 元
97,滑鼠,羅技 M190 無線滑鼠-黑,"['◆全尺寸的舒適感', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']",349 元
99,滑鼠,羅技 M190 無線滑鼠-紅,"['◆全尺寸的舒適感', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']",349 元
98,滑鼠,羅技 M190 無線滑鼠-藍,"['◆全尺寸的舒適感。', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']",349 元


In [22]:
distances

array([[183.30498, 191.29105, 215.88385, 221.04794, 236.49658]],
      dtype=float32)

In [23]:
query = "200元內的羅技滑鼠"
results, distances = search_products(query)
results

,分類,商品名稱,描述,價格
94,滑鼠,羅技 G102 炫彩遊戲滑鼠,"['◆七彩光學效果，讓遊戲更充滿樂趣', '◆高達8000DPI遊戲等級感應器', '◆機械...",499 元
100,滑鼠,羅技 M90 光學滑鼠,"['◆羅技 logi M90', '◆全尺寸左右手皆適用', '◆有線']",189 元
96,滑鼠,羅技 M100r 光學滑鼠 有線設計 雙手均適用 1000 dpi高解析度,"['◆有線設計，免除更換電池的過程', '◆雙手均適用的舒適設計', '◆1000 dpi高...",299 元
102,滑鼠墊,darkFlash Flex800滑鼠墊 產品尺寸：80*30 cm 厚度：4mm 厚度舒適...,"['◆darkFlash Flex800 mousepad', '◆天然橡膠材質加厚，兼具外...",499 元
97,滑鼠,羅技 M190 無線滑鼠-黑,"['◆全尺寸的舒適感', '◆無延遲的無線連線。', '◆精確追蹤/輕鬆捲動。']",349 元


In [24]:
distances

array([[230.57358, 245.57944, 251.47884, 254.30429, 256.75046]],
      dtype=float32)